In [1]:
from threeML import *
from HAWCpyLike import HAWCpyLike
import region_of_interest

%matplotlib notebook

Configuration read from /home/giacomov/.threeML/threeML_config.yml
Plotter is MatPlotlib


In [14]:
# ra_crab, dec_crab = 83.633083, 22.014500

# roi = region_of_interest.HealpixConeROI(5.0, ra=ra_crab, dec=dec_crab)

# hawc = HAWCpyLike("HAWC", 
#                   "/home/giacomov/science/hawc/data/maptree_1024.root", 
#                   "/home/giacomov/science/hawc/data/response.root",
#                  roi)

ra_crab, dec_crab = 100.0, 22.0

roi = region_of_interest.HealpixConeROI(5.0, ra=ra_crab, dec=dec_crab)

hawc = HAWCpyLike("HAWC", 
                  "/home/giacomov/develop/hawc-docker/simulated_data/maptree_256.root", 
                  "/home/giacomov/develop/hawc-docker/simulated_data/detector_response.root",
                 roi)

hawc.set_active_measurements(1, 9)

hawc.display()

Response: 
----------

Response file: /home/giacomov/develop/hawc-docker/simulated_data/detector_response.root
Number of dec bins: 106
Number of energy/nHit planes per dec bin: 10

Map Tree: 
----------



,Bin,Nside,Scheme,Obs counts,Bkg counts,obs/bkg,Pixels in ROI,Area (deg^2)
0,0,256,RING,1.232866e+09,1.232804e+09,1.000050,1492,78.264132
1,1,256,RING,8.648596e+07,8.642666e+07,1.000686,1492,78.264132
2,2,256,RING,2.761620e+07,2.757502e+07,1.001493,1492,78.264132
3,3,256,RING,7.482597e+06,7.466526e+06,1.002152,1492,78.264132
4,4,256,RING,9.398600e+05,9.324169e+05,1.007983,1492,78.264132
5,5,256,RING,2.063070e+05,2.034786e+05,1.013900,1492,78.264132
6,6,256,RING,3.691200e+04,3.612566e+04,1.021767,1492,78.264132
7,7,256,RING,2.179900e+04,2.123331e+04,1.026642,1492,78.264132
8,8,256,RING,5.891000e+03,5.651473e+03,1.042383,1492,78.264132
9,9,256,RING,8.319000e+03,8.042094e+03,1.034432,1492,78.264132


This Map Tree contains 508.195 transits
Total data size: 0.11936 Mbyte

Active energy/nHit planes: 
---------------------------

[1, 2, 3, 4, 5, 6, 7, 8, 9]


In [15]:
# spectrum = Log_parabola()

# source = PointSource("CrabNebula", ra=ra_crab, dec=dec_crab, spectral_shape=spectrum)

# # NOTE: if you use units, you have to set up the values for the parameters
# # AFTER you create the source, because during creation the function Log_parabola
# # gets its units

# spectrum.piv = 10 * u.TeV  # Pivot energy
# spectrum.piv.fix = True

# spectrum.K = 1e-14 / (u.TeV * u.cm**2 * u.s)  # norm (in 1/(keV cm2 s))
# spectrum.K.bounds = (1e-25, 1e-19) # without units energies are in keV

# spectrum.beta = 0  # log parabolic beta
# spectrum.beta.bounds = (-4., 2.)

# spectrum.alpha = -2.5  # log parabolic alpha (index)
# spectrum.alpha.bounds = (-4., 2.)

# Define the spectral and spatial models for the source
spectrum = Cutoff_powerlaw()
source = PointSource("TestSource", ra=100.0, dec=22.0, spectral_shape=spectrum)

spectrum.K = 3.15e-11 / (u.TeV * u.cm ** 2 * u.s)
spectrum.K.bounds = (1e-22, 1e-18)  # without units energies are in keV

spectrum.piv = 1 * u.TeV
spectrum.piv.fix = True

spectrum.index = -2.37
spectrum.index.bounds = (-4, -1)

spectrum.xc = 42.3 * u.TeV
spectrum.xc.bounds = (1 * u.TeV, 100 * u.TeV)


model = Model(source)

data = DataList(hawc)

jl = JointLikelihood(model, data)

Using dec bin 57 for source TestSource


In [16]:
hawc._convolved_point_sources.size

<Quantity 0.11936 Mbyte>

In [18]:
hawc.get_log_like()

#-310426.9997955779

-61963.576584007118

In [19]:
jl.set_minimizer("ROOT")
_ = jl.fit()

3.15e-11,2.37,42.3

Best fit values:




WARNING UserWarning: 14.58 percent of samples have been thrown away because they failed the constraints on the parameters. This results might not be suitable for error propagation. Enlarge the boundaries until you loose less than 1 percent of the samples.



,result,unit
parameter,,
TestSource.spectrum.main.Cutoff_powerlaw.K,(3.46 -0.20 +0.22) x 10^-19,1 / (cm2 keV s)
TestSource.spectrum.main.Cutoff_powerlaw.index,-2.470 +/- 0.04,
TestSource.spectrum.main.Cutoff_powerlaw.xc,(8.6 -1.2 +1.4) x 10^10,keV



Correlation matrix:



1.00,-0.95,0.76
-0.95,1.00,-0.92
0.76,-0.92,1.00



Values of -log(likelihood) at the minimum:



,-log(likelihood)
HAWC,57461.633084
total,57461.633084



Values of statistical measures:



,statistical measures
AIC,114921.266168
BIC,114923.266168


In [21]:
print spectrum(1.0 * u.TeV).to(1/(u.TeV * u.cm**2 * u.s)) / 6

5.70665904381e-11 1 / (cm2 s TeV)


In [9]:
print 1.49499587383e-10 / 2.37224580333e-11

6.30202768925


In [12]:
import healpy as hp

print 1 / hp.nside2pixarea(1024)

1001316.32165
